In [ ]:
base_model_name = "unsloth/SmolLM2-360M-Instruct-GGUF" #"unsloth/Llama-3.2-3B-bnb-4bit"
target_model_name = "unsloth/SmolLM2-360M-Instruct-GGUF"
model_name = "lucaelin/llama-3.2-3b-instruct-fc"

dataset_name = "lucaelin/covas_next"

max_sequence_length = 16*1024
lora_rank = 32
lora_alpha = 8
learning_rate = 1e-4
weight_decay = 0.01

instruction_template = "<|im_start|>user\n"
response_template = "<|im_start|>assistant\n"
chat_template = "{% set loop_messages = messages %}{% for message in loop_messages %}{% set role = message['role'] %}{% if 'tool_calls' in message %}{% set text = '<tool_call>' + message['tool_calls'][0]['function']|tojson + '</tool_call>' %}{% endif %}{% if 'content' in message %}{% set text = message['content'] %}{% endif %}{% if loop.index0 == 0 and tools is defined %}{% set text = message['content'] + '\n\nYou have access to the following tools:\n<tools>\n' + tools|tojson + '\n</tools>' %}{% endif %}{% set content = '<|im_start|>' + role + '\n'+ text | trim + '<|im_end|>' %}{% if loop.index0 == 0 %}{% set content = bos_token + content %}{% endif %}{{ content }}{% endfor %}{{ '<|im_start|>assistant\n' }}"

In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
from unsloth import FastLanguageModel
from transformers import AutoTokenizer
import torch
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = base_model_name, # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_sequence_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

tokenizer = AutoTokenizer.from_pretrained(target_model_name)

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = lora_alpha,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = True,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

<a name="Data"></a>
### Data Prep
We now use the `Llama-3.1` format for conversation style finetunes. We use [Maxime Labonne's FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset in ShareGPT style. But we convert it to HuggingFace's normal multiturn format `("role", "content")` instead of `("from", "value")`/ Llama-3 renders multi turn conversations like below:

```
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Hello!<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hey there! How are you?<|eot_id|><|start_header_id|>user<|end_header_id|>

I'm great thanks!<|eot_id|>
```

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3` and more.

In [ ]:
from unsloth.chat_templates import get_chat_template
import json

tokenizer.chat_template = chat_template

def formatting_prompts_func(example):
    messages = json.loads(example["messages"])
    tools = json.loads(example["tools"])
    text = tokenizer.apply_chat_template(messages, tools=tools, tokenize = False, add_generation_prompt = False)
    return { "text" : text, }
pass

from datasets import load_dataset
dataset = load_dataset(dataset_name, split = "train")

from unsloth.chat_templates import standardize_sharegpt
dataset = dataset.map(formatting_prompts_func, batched = False,)

In [ ]:
dataset[5]["messages"]

In [ ]:
dataset[5]["text"]

In [ ]:
# Shuffle the dataset with a fixed seed
dataset = dataset.shuffle(seed=42)

# Split the dataset into 95% training and 5% validation
split_dataset = dataset.train_test_split(test_size=0.01, seed=42)
dataset = split_dataset['train']
validation_dataset = split_dataset['test']
print(f"Training dataset size: {len(dataset)}")
print(f"Validation dataset size: {len(validation_dataset)}")

<a name="Train"></a>
### Train the model

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = validation_dataset,
    dataset_text_field = "text",
    max_seq_length = max_sequence_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 8,
        per_device_eval_batch_size = 8,
        gradient_accumulation_steps = 8,
        warmup_steps = 5,
        num_train_epochs = 3, # Set this for 1 full training run.
        #max_steps = 10,
        eval_steps = 50,
        eval_strategy = "steps",
        learning_rate = learning_rate,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = weight_decay,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs.

In [ ]:
from unsloth.chat_templates import train_on_responses_only

trainer = train_on_responses_only(
    trainer,
    instruction_part = instruction_template,
    response_part = response_template,
)

We verify masking is actually done:

In [ ]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

In [ ]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

<a name="Memory Stats"></a>
### Memory Stats

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
trainer_stats = trainer.train()

In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

<a name="Inference"></a>
### Inference
Use the validation dataset to check model performance

We use `min_p = 0.1` and `temperature = 1.5`. Read this [Tweet](https://x.com/menhguin/status/1826132708508213629) for more information on why.

In [ ]:
FastLanguageModel.for_inference(model)
# Pick an example from the validation dataset
example = validation_dataset[0]

# Print the ground truth
print("Ground Truth:")
print(example["text"])

# Extract the text until the response part
instruction_part = example["text"].split(response_template)[0]

# Prepare the input for inference
inputs = tokenizer(
    instruction_part,
    return_tensors="pt",
    add_special_tokens=False,
).to("cuda")

# Run the inference
from transformers import TextStreamer
print("\nInference Result:")
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

<a name="Save"></a>
### Saving, loading finetuned models

Save the final model as LoRA adapters locally and publish to hub.

**[NOTE]** We use a trick here and rewrite the base model name so that it points to the target model instead. This increases model performance somehow.

In [ ]:
model.active_peft_config.base_model_name_or_path = target_model_name
model.config.update({"base_model_name_or_path": target_model_name})
model.save_pretrained(model_name) # Local saving
tokenizer.save_pretrained(model_name)
model.push_to_hub(model_name+'-lora') # Online saving
tokenizer.push_to_hub(model_name+'-lora') # Online saving

Next, unload the trained model to free up resource for model merge and export 

In [ ]:
import gc
import torch

del model
gc.collect()
torch.cuda.empty_cache()

Now load the model config we just saved which applies our LoRA adapter the target_model instead:

In [ ]:
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,

    max_seq_length = max_sequence_length,
    dtype = dtype,
    load_in_4bit = False,
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

In [ ]:
# Pick an example from the validation dataset
example = validation_dataset[1]

# Print the ground truth
print("Ground Truth:")
print(example["text"])

# Extract the text until the response part
instruction_part = response_template.join(example["text"].split(response_template)[0:1])

# Prepare the input for inference
inputs = tokenizer(
    instruction_part,
    return_tensors="pt",
    add_special_tokens=False,
).to("cuda")

# Run the inference
from transformers import TextStreamer
print("\nInference Result:")
print(instruction_part)
print('---')
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion

In [ ]:
# Save to multiple GGUF options
model.push_to_hub_gguf(
    model_name+'-gguf',
    tokenizer,
    quantization_method = ["q4_k_m", "q8_0"],
)